In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# %pip install tabulate

# Outage Severity and People Affected

**Name(s)**: Quy-Dzu Do

**Website Link**: https://krazykats.github.io/Outages_and_Stats/

In [3]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from dsc80_utils import * # Feel free to uncomment and use this.

LOCAL_DIR = os.getcwd()
website_folder = os.path.join(LOCAL_DIR, "..", "Website_Resources")

In [4]:
from final_proj import *

## Step 1: Introduction

I chose to do the Outage Dataset as I felt the data to be more relevant to me as I hav lived in California for most of my life have gone through many outages due to High Winds, fires and other reasons. The League data does not interest me since I have no experience with the game so I have no expertise in the field and any conclusions I couls draw would most likely lack context needed to properly extrapolate the data. As for the recepies, I would not be opposed to working with it but I find the subject of outages to be more compelling than data on cooking and nutritional facts. 

For the outages dataset, I am most interested in looking at the relation between frequency and duration of outages with the corresponding population affected. I expect there to be a difference as most companies would be more concerned with population centers and centers of commerce getting an outage than a rural population but I would like to see how disproporionate is the result. 

## Step 2: Data Cleaning and Exploratory Data Analysis

We shall begin by pulling the data into a Pandas Dataframe skipping the first 5 rows and the seventh row as they do not contain data and we set the ```"OBV"``` Column to the index as it IDs the rows of data. We will also convert the ```"OUTAGE.START.TIME"``` and ```"OUTAGE.RESTORATION.TIME"``` to timedelta objects, and ```"OUTAGE.START.DATE"``` and ```"OUTAGE.RESTORATION.DATE"``` to timestamp objects so they can be combined into a single date-time value stores as a Pandas Timestamp in a new dataframe with ```"OUTAGE.START.DATE"``` and ```"OUTAGE.RESTORATION.DATE"``` containing the new objects and the Time columns dropped.

In [5]:
data_set = os.path.join(LOCAL_DIR, "outage.xlsx")
data = pd.read_excel(data_set, skiprows=lambda x: x in [0,1,2,3,4,6],
                     dtype={"OUTAGE.START.TIME": str,
                            "OUTAGE.RESTORATION.TIME": str})
data = data.drop("variables", axis=1).set_index("OBS")
data_time= time_to_datetime(data)
data_time.head()

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,...,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
OBS,,,,,,,,,
1,2011,7.0,Minnesota,MN,...,0.6,91.59,8.41,5.48
2,2014,5.0,Minnesota,MN,...,0.6,91.59,8.41,5.48
3,2010,10.0,Minnesota,MN,...,0.6,91.59,8.41,5.48
4,2012,6.0,Minnesota,MN,...,0.6,91.59,8.41,5.48
5,2015,7.0,Minnesota,MN,...,0.6,91.59,8.41,5.48


In [6]:
print(data_time.head().to_markdown(index=False))

|   YEAR |   MONTH | U.S._STATE   | POSTAL.CODE   | NERC.REGION   | CLIMATE.REGION     |   ANOMALY.LEVEL | CLIMATE.CATEGORY   | OUTAGE.START.DATE   | OUTAGE.START.TIME   | OUTAGE.RESTORATION.DATE   | OUTAGE.RESTORATION.TIME   | CAUSE.CATEGORY     | CAUSE.CATEGORY.DETAIL   |   HURRICANE.NAMES |   OUTAGE.DURATION |   DEMAND.LOSS.MW |   CUSTOMERS.AFFECTED |   RES.PRICE |   COM.PRICE |   IND.PRICE |   TOTAL.PRICE |   RES.SALES |   COM.SALES |   IND.SALES |   TOTAL.SALES |   RES.PERCEN |   COM.PERCEN |   IND.PERCEN |   RES.CUSTOMERS |   COM.CUSTOMERS |   IND.CUSTOMERS |   TOTAL.CUSTOMERS |   RES.CUST.PCT |   COM.CUST.PCT |   IND.CUST.PCT |   PC.REALGSP.STATE |   PC.REALGSP.USA |   PC.REALGSP.REL |   PC.REALGSP.CHANGE |   UTIL.REALGSP |   TOTAL.REALGSP |   UTIL.CONTRI |   PI.UTIL.OFUSA |   POPULATION |   POPPCT_URBAN |   POPPCT_UC |   POPDEN_URBAN |   POPDEN_UC |   POPDEN_RURAL |   AREAPCT_URBAN |   AREAPCT_UC |   PCT_LAND |   PCT_WATER_TOT |   PCT_WATER_INLAND |
|-------:|--------:|:-------

In [44]:
fig1 = data_time["OUTAGE.DURATION"]\
    .hist(bins=100, title="Outage Duration Distribution", labels={"value": "Duration (minutes)", "index": "Frequency"})
fig1.show()

In [45]:
fig2 = data_time.loc[data_time["U.S._STATE"] == "California", "OUTAGE.DURATION"]\
    .hist(bins=100, title="Outage Duration Distribution in California", labels={"value": "Duration (minutes)", "index": "Frequency"})
fig2.show()

In [46]:
fig1.write_html(os.path.join(website_folder, "uni_1_1.html"), include_plotlyjs='cdn')
fig2.write_html(os.path.join(website_folder, "uni_1_2.html"), include_plotlyjs='cdn')

In [49]:
fig = data_time.groupby("YEAR")["OUTAGE.DURATION"].agg("mean")\
    .plot.line(title="Average Outage Duration by Year")
fig.show()

In [50]:
fig.write_html(os.path.join(website_folder, "uni_2_1.html"), include_plotlyjs='cdn')

In [51]:
# Create the scatter plot
fig = px.scatter(data_time, x='OUTAGE.DURATION',
                 y='CUSTOMERS.AFFECTED',
                 title="Customers Affected vs. Outage Duration",
                 labels={'OUTAGE.DURATION': 'Outage Duration (minutes)',
                         'CUSTOMERS.AFFECTED': 'Customers Affected'})

# Show the plot
fig.show()

In [52]:
fig.write_html(os.path.join(website_folder, "bi_1.html"), include_plotlyjs='cdn')

In [53]:
# Create the scatter plot
fig = px.scatter(data_time, x='TOTAL.CUSTOMERS',
                 y='CUSTOMERS.AFFECTED',
                 title="Customers Affected vs. Total Customers",
                 labels={'TOTAL.CUSTOMERS': 'Number of Customers in the State',
                         'CUSTOMERS.AFFECTED': 'Customers Affected'})

# Show the plot
fig.show()

In [54]:
fig.write_html(os.path.join(website_folder, "bi_2.html"), include_plotlyjs='cdn')

In [ ]:
table = pd.pivot_table(data_time, values=['OUTAGE.DURATION',
                                          "CUSTOMERS.AFFECTED",
                                          "DEMAND.LOSS.MW"],
                         index=["U.S._STATE"],
                        aggfunc="mean")
table

,CUSTOMERS.AFFECTED,DEMAND.LOSS.MW,OUTAGE.DURATION
U.S._STATE,,,
Alabama,94328.80,291.50,1152.80
Alaska,14273.00,35.00,NaN
Arizona,64402.67,1245.70,4552.92
...,...,...,...
West Virginia,179794.33,362.00,6979.00
Wisconsin,45876.00,161.00,7904.11
Wyoming,11833.33,26.75,33.33


In [ ]:
print(table.to_markdown(index=False))

## Step 3: Assessment of Missingness

In [13]:
missing_data = data_time.copy()
missing_data["MISSING_LABEL"] = (missing_data["OUTAGE.DURATION"].isna()).astype(str)
missing_data


,YEAR,MONTH,U.S._STATE,POSTAL.CODE,...,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,MISSING_LABEL
OBS,,,,,,,,,
1,2011,7.0,Minnesota,MN,...,91.59,8.41,5.48,False
2,2014,5.0,Minnesota,MN,...,91.59,8.41,5.48,False
3,2010,10.0,Minnesota,MN,...,91.59,8.41,5.48,False
...,...,...,...,...,...,...,...,...,...
1532,2009,8.0,South Dakota,SD,...,98.31,1.69,1.69,False
1533,2009,8.0,South Dakota,SD,...,98.31,1.69,1.69,False
1534,2000,NaN,Alaska,AK,...,85.76,14.24,2.90,True


In [14]:
stats, obs = permutation_test(missing_data, 'MONTH', 'MISSING_LABEL', tvd)
np.mean(stats >= obs)

np.float64(0.123)

In [15]:
fig = px.histogram(stats)
fig.add_vline(x=obs, line_width=3, line_dash="dash", line_color="red")
fig.show()

In [16]:
fig.write_html(os.path.join(website_folder, "missing_MCAR.html"), include_plotlyjs='cdn')

In [17]:
stats, obs = permutation_test(missing_data, 'NERC.REGION', 'MISSING_LABEL', tvd)
np.mean(stats >= obs)

np.float64(0.0)

In [18]:
obs

np.float64(0.3153910849453322)

In [19]:
fig = px.histogram(stats)
fig.add_vline(x=obs, line_width=3, line_dash="dash", line_color="red")
fig.show()

In [20]:
fig.write_html(os.path.join(website_folder, "missing_MAR.html"), include_plotlyjs='cdn')

In [21]:
missing_data

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,...,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,MISSING_LABEL
OBS,,,,,,,,,
1,2011,7.0,Minnesota,MN,...,91.59,8.41,5.48,False
2,2014,5.0,Minnesota,MN,...,91.59,8.41,5.48,False
3,2010,10.0,Minnesota,MN,...,91.59,8.41,5.48,False
...,...,...,...,...,...,...,...,...,...
1532,2009,8.0,South Dakota,SD,...,98.31,1.69,1.69,False
1533,2009,8.0,South Dakota,SD,...,98.31,1.69,1.69,False
1534,2000,NaN,Alaska,AK,...,85.76,14.24,2.90,True


## Step 4: Hypothesis Testing

In [22]:
permutation_data = data_time.copy()
permutation_data["Is_California"] = (permutation_data["U.S._STATE"] == "California").astype(str)


In [23]:
diff_medians(permutation_data, "OUTAGE.DURATION", "Is_California")

np.float64(-581.5)

In [24]:
n = 1000
medians_diff = []
observed_diff = diff_medians(permutation_data, "OUTAGE.DURATION", "Is_California")
for _ in range(n):
    permutation_data["shuffled_labels"] = np.random.permutation(permutation_data["Is_California"])
    medians_diff.append(diff_medians(permutation_data, "OUTAGE.DURATION", "shuffled_labels"))

np.mean([diff <= observed_diff for diff in medians_diff])

np.float64(0.0)

## Step 5: Framing a Prediction Problem

While Outage Duration is a good classifier for how extreme an outage is, most companies would be more interested in the effects it has on customers and how whether they are more likely to complain. Thus, we will be using the given data and predicting how many customers are affected. This will help the companies to identify events that are more likely to affect more people. From there, the companies may seek to create new methods to counter act on these specific predictive variables. 

Looking at our basic model, we can see that much or data seems very much skewed by extremely low values that happen very often and a few outlier high values. This has resulted in very high RMSE values and overall a bad predictor for what is creates a high amount of affected customers. To remedy this, we will chage from regression to classification and define a new variable "High_Risk_Customers" as a Binary classification of whether a certain event will have 

## Step 6: Baseline Model

In [25]:
base_pred_model = data_time.copy()
base_pred_model = base_pred_model[["NERC.REGION",
                                   "MONTH",
                                   "CLIMATE.REGION",
                                   "CLIMATE.CATEGORY",
                                   "ANOMALY.LEVEL",
                                   "CAUSE.CATEGORY",
                                   "RES.PERCEN",
                                   "COM.PERCEN",
                                   "IND.PERCEN",
                                   "RES.CUSTOMERS",
                                   "COM.CUSTOMERS",
                                   "IND.CUSTOMERS",
                                   "POPULATION",
                                   "POPPCT_URBAN",
                                   "CUSTOMERS.AFFECTED"]]
base_pred_model.dropna(inplace=True)
base_pred_model["CUSTOMERS.AFFECTED"] = base_pred_model["CUSTOMERS.AFFECTED"].apply(lambda x: int(x>150_000))

In [26]:
base_pred_model["CUSTOMERS.AFFECTED"].sum()

np.int64(261)

In [27]:
data_time.plot.scatter(x="RES.PERCEN", y="CUSTOMERS.AFFECTED")

In [28]:
data_time.plot.scatter(x="ANOMALY.LEVEL", y="CUSTOMERS.AFFECTED")

In [29]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))


from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import random
random.seed(10)

X = base_pred_model.drop("CUSTOMERS.AFFECTED", axis=1)
y = base_pred_model["CUSTOMERS.AFFECTED"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

model = DecisionTreeClassifier(max_depth = 4)
log_transformer = FunctionTransformer(np.log)
exp_transformer = FunctionTransformer(np.exp)
square_transformer = FunctionTransformer(np.square)
preproc = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['NERC.REGION',
                                  'MONTH', 'CLIMATE.REGION',
                                  'CLIMATE.CATEGORY', 'CAUSE.CATEGORY'])
    ],
    remainder='passthrough',
    force_int_remainder_cols=False,
)

pipe = Pipeline([('preproc', preproc),
                  ('model', model)])

pipe.fit(X_train, y_train)
(pipe.predict(X_test) == y_test).mean()

np.float64(0.7052238805970149)

In [30]:
y_pred = pipe.predict(X_test)

TP = 0
FP = 0

for i in range (len(y_test)):
    if y_pred[i] == 1:
        if y_test.iloc[i] == 1:
            TP += 1
        else:
            FP += 1

TP/(TP+FP)

0.2631578947368421

## Step 7: Final Model

For this final Model, let us try to use a SVC model and see it can do better than our base model. We will apply some column transformations to some of the columns as there may be some information to gain by changing them. One of the columns changed is the Anomoly Level which is now squared. This is due to the extremes of the anaomoly level not having any high values and thus are squared so that there can be an easier split in the middle rather than at the sides where there might need to be multiple splits. We will also apply an exponential transformer to separate the higher values of the percentages which are where there are more high customers affected but there is still a lot of low class points there too so we are hoping to introduce changes within that axis so that there are greater distances in the higher values are more spread and thus will allow the support vector to split the data at a higher value so that we can get more True Positives and less False Positives.

We will also search along the tolerance levels of the model to observe what tolerances fit the model the best and to see if increasing the tolerance will affect the models performance

In [31]:
log_transformer = FunctionTransformer(np.log)
exp_transformer = FunctionTransformer(np.exp)
square_transformer = FunctionTransformer(np.square)
preproc = ColumnTransformer(
    transformers=[
        ('num', exp_transformer, ['RES.PERCEN']),
        ('square', square_transformer, ['ANOMALY.LEVEL']),
        ('cat', OneHotEncoder(), ['NERC.REGION',
                                  'MONTH', 'CLIMATE.REGION',
                                  'CLIMATE.CATEGORY', 'CAUSE.CATEGORY'])
    ],
    remainder='passthrough',
    force_int_remainder_cols=False,
)

hyper_param = {}

for j in range(1000):
    model = SVC(degree = 3, gamma = 'auto', tol = 1e-3 * (1 + 50*j))
    pipe = Pipeline([('preproc', preproc),
                      ('model', model)])

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)

    y_pred = pipe.predict(X_test)

    TP = 0
    FP = 0

    for i in range (len(y_test)):
        if y_pred[i] == 1:
            if y_test.iloc[i] == 1:
                TP += 1
            else:
                FP += 1

    hyper_param[j] = ((pipe.predict(X_test) == y_test).mean() , TP/(TP+FP))

hyper_param

{0: (np.float64(0.75), 0.5882352941176471),
 1: (np.float64(0.75), 0.5882352941176471),
 2: (np.float64(0.75), 0.5882352941176471),
 3: (np.float64(0.75), 0.5882352941176471),
 4: (np.float64(0.75), 0.5882352941176471),
 5: (np.float64(0.75), 0.5882352941176471),
 6: (np.float64(0.75), 0.5882352941176471),
 7: (np.float64(0.75), 0.5882352941176471),
 8: (np.float64(0.75), 0.5882352941176471),
 9: (np.float64(0.75), 0.5882352941176471),
 10: (np.float64(0.75), 0.5882352941176471),
 11: (np.float64(0.75), 0.5882352941176471),
 12: (np.float64(0.75), 0.5882352941176471),
 13: (np.float64(0.75), 0.5882352941176471),
 14: (np.float64(0.75), 0.5882352941176471),
 15: (np.float64(0.75), 0.5882352941176471),
 16: (np.float64(0.75), 0.5882352941176471),
 17: (np.float64(0.75), 0.5882352941176471),
 18: (np.float64(0.75), 0.5882352941176471),
 19: (np.float64(0.75), 0.5882352941176471),
 20: (np.float64(0.75), 0.5882352941176471),
 21: (np.float64(0.75), 0.5882352941176471),
 22: (np.float64(0.7

In [32]:
# Final model

model = SVC(degree = 3, gamma = 'auto', tol = 1e-3)
pipe = Pipeline([('preproc', preproc),
                      ('model', model)])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

TP = 0
FP = 0

for i in range (len(y_test)):
    if y_pred[i] == 1:
        if y_test.iloc[i] == 1:
            TP += 1
        else:
            FP += 1

TP/(TP+FP)

0.5882352941176471

## Step 8: Fairness Analysis

Let us now look at the fairness of the model. To do this we will be looking at the if the model predicts similarly across region in the US. We will divide the data set into 2 lables, "True" for states in the West and "False" for the States in the East and then run a permutation test seeing if the difference in the precision is statistically different between the 2 groups.

In [33]:
# prep the data set
base_pred_model = data_time.copy()
base_pred_model = base_pred_model[["U.S._STATE",
                                   "NERC.REGION",
                                   "MONTH",
                                   "CLIMATE.REGION",
                                   "CLIMATE.CATEGORY",
                                   "ANOMALY.LEVEL",
                                   "CAUSE.CATEGORY",
                                   "RES.PERCEN",
                                   "COM.PERCEN",
                                   "IND.PERCEN",
                                   "RES.CUSTOMERS",
                                   "COM.CUSTOMERS",
                                   "IND.CUSTOMERS",
                                   "POPULATION",
                                   "POPPCT_URBAN",
                                   "CUSTOMERS.AFFECTED"]]
base_pred_model.dropna(inplace=True)
base_pred_model["CUSTOMERS.AFFECTED"] = base_pred_model["CUSTOMERS.AFFECTED"].apply(lambda x: int(x>150_000))

In [34]:
# Categorize the states as in West or not
base_pred_model["Is_West_Missippi"] = (base_pred_model["U.S._STATE"].isin([
                                                                   "Alaska",
                                                                   "Arizona",
                                                                   "Arkansas",
                                                                   "California",
                                                                   "Colorado",
                                                                   "Hawaii",
                                                                   "Idaho",
                                                                   "Iowa",
                                                                   "Kansas",
                                                                   "Louisiana",
                                                                   "Minnesota",
                                                                   "Missouri",
                                                                   "Montana",
                                                                   "Nebraska",
                                                                   "Nevada",
                                                                   "New Mexico",
                                                                   "North Dakota",
                                                                   "Oklahoma",
                                                                   "Oregon",
                                                                   "South Dakota",
                                                                   "Texas",
                                                                   "Utah",
                                                                   "Washington",
                                                                   "Wyoming"
                                                                  ]))
base_pred_model.drop("U.S._STATE", axis=1, inplace=True)

In [35]:
X_data = base_pred_model.drop("CUSTOMERS.AFFECTED", axis=1)
y_data = base_pred_model["CUSTOMERS.AFFECTED"]

In [36]:
def split_data(X, y):
    True_data_X = X[X["Is_West_Missippi"] == True]
    True_data_y = y[X["Is_West_Missippi"] == True]
    False_data_X = X[X["Is_West_Missippi"] == False]
    False_data_y = y[X["Is_West_Missippi"] == False]
    return True_data_X, True_data_y, False_data_X, False_data_y

def get_precision (y_test, y_pred):
    TP = 0
    FP = 0

    for i in range (len(y_test)):
        if y_pred[i] == 1:
            if y_test.iloc[i] == 1:
                TP += 1
            else:
                FP += 1
    if TP == 0 and FP == 0:
        return 0

    return TP/(TP+FP)

def find_diff_precision(True_data_X, True_data_y, False_data_X, False_data_y):
    True_data_y_pred = pipe.predict(True_data_X)
    False_data_y_pred = pipe.predict(False_data_X)

    True_data_precision = get_precision(True_data_y, True_data_y_pred)
    False_data_precision = get_precision(False_data_y, False_data_y_pred)

    return abs(True_data_precision - False_data_precision)



True_data_X, True_data_y, False_data_X, False_data_y = split_data(X_data, y_data)
observed_diff = find_diff_precision(True_data_X, True_data_y, False_data_X, False_data_y)
observed_diff

0.02946650124069483

In [37]:
# Premutation test

permutation_inner_data = X_data.copy()
stats = []

for _ in range(100):
    permutation_inner_data["Is_West_Missippi"] = \
        np.random.permutation(permutation_inner_data["Is_West_Missippi"])

    True_data_X, True_data_y, False_data_X, False_data_y = \
        split_data(permutation_inner_data, y_data)

    stats.append(find_diff_precision(True_data_X,
                                     True_data_y,
                                     False_data_X,
                                     False_data_y))

(np.array(stats) > observed_diff).mean()

np.float64(0.58)

In [38]:
fig = px.histogram(stats, labels={"value": "Difference in Precision",
                                  "count": "Frequency"})
fig.add_vline(x=observed_diff, line_width=3, line_dash="dash", line_color="red")
fig.show()

In [39]:
fig.write_html(os.path.join(website_folder, "fairness_analysis.html"), include_plotlyjs='cdn')
